In [1]:
import sqlite3
import pandas as pd
import MeCab
import ipadic
import re
import sys
sys.path.append("../chat")
import openai
import os

In [2]:
import chat
from chat.db import query
from chat.entry_points.load_keywords import condition
from chat.chunk import morphological_analysis

In [3]:
def question2keywords(s: str) -> list[str]:
    ret = [
        w.word for w in 
        morphological_analysis(s)
        if condition(w.word, repr(w.attributes))
    ]    
    return ret

In [4]:
def keywords2simirality(ws: list[str]) -> pd.DataFrame:
    words = ",".join(map(lambda w: f"'{w}'", ws))
    sql = f"""
    with temp1 as (
        select
          *
        from
          tfidfs
        where
          word in ({words})
    )
    select
      chunk_id,
      sum(tfidf) as similarity
    from
      temp1
    group by
      chunk_id
    """
    return query(sql).sort_values("similarity", ascending=False)

In [5]:
def load_content(ids: list[int]) -> list[str]:
    sql = f"""
    select
      content
    from
      chunks
    where
      id in ({",".join(map(str, ids))})
    """
    df = query(sql)
    return list(df["content"])

In [6]:
def make_prompt(question: str) -> str:
    kws = question2keywords(question)
    sim = keywords2simirality(kws)
    context = "\n".join(load_content(list(sim.head(5)["chunk_id"])))
    template = f"""
以下に、日本の金融機関に関する説明文があります。また、説明文に続いて、質問文があります。この説明文の情報から質問文に答えてください。
説明文
-----------
{context}
-----------
質問文
-----------
{question}
    """
    return template

In [7]:
def ask2chatgpt(question: str, print_answer=False):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    content = make_prompt(question)

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": content}
      ]
    )
    
    if print_answer:
        print(completion.choices[0].message["content"])
    
    return completion

In [9]:
_ = ask2chatgpt("ふくおかフィナンシャルグループのマネーサービスについて教えて下さい", True)

query: reading cache
query: reading cache


回答：残念ながら、説明文にはふくおかフィナンシャルグループのマネーサービスについての情報は含まれていません。


In [10]:
_ = ask2chatgpt("ふくおかフィナンシャルグループのアプリについて教えて下さい", True)

query: reading cache
query: reading cache


説明文にはふくおかフィナンシャルグループのアプリに関する情報が含まれていないため、回答できません。申し訳ありません。


In [ ]:
_ = ask2chatgpt("ふくおかフィナンシャルグループの中期経営計画を教えて下さい", True)

query: reading cache
query: reading cache


In [122]:
_ = ask2chatgpt("ふくおかフィナンシャルグループのSDGsに関する取り組みについて教えて下さい", True)

ふくおかフィナンシャルグループは、SDGsへの取組みの一環として、福岡銀行、熊本銀行、十八親和銀行で脱炭素応援私募債の取扱いを開始し、発行されたお客様から頂く収益の一部を、全国の被災した森林や開発跡地での森林再生活動を行う認定NPO法人環境リレーションズ研究所へ寄付する商品を提供しています。また、全国の地域金融機関との連携を加速させ、お客さまのサステナビリティ経営実現に向けた取り組みを支援していることが説明文から読み取れます。


In [123]:
_ = ask2chatgpt("ふくおかフィナンシャルグループと横浜銀行は、どういう関係がありますか", True)

ふくおかフィナンシャルグループと横浜銀行はビジネス上のパートナーシップはありますが、何らかの所有関係や出資関係は示されていません。横浜銀行は、ふくおかフィナンシャルグループのグループ会社である株式会社サステナブルスケールと協力し、SDGsESGスコアリング評価サービス「SustainableScaleIndex」を横浜銀行の顧客向けに提供しています。
